In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
test.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt
0,1,9,1800,Gd,2,702,Ex,1800,1800,Ex,2,2007,2007,2007
1,2,6,1082,TA,1,240,TA,1082,1082,TA,1,1948,1950,1948
2,3,6,1573,Gd,2,440,Gd,756,769,Gd,2,2000,2000,2000
3,4,6,2443,Gd,3,744,Gd,1158,1158,Gd,2,2004,2004,2004
4,5,5,1040,TA,2,686,TA,1040,1040,TA,1,1968,1968,1991


In [5]:
test.drop('id',axis=1,inplace=True)

In [6]:
train.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


In [7]:
train.drop('id',axis=1 , inplace=True)

In [8]:
print(train.shape,test.shape)

(1350, 14) (1350, 13)


이상치 변경

In [9]:
train.loc[254,'Garage Yr Blt'] = 2007

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
qual_cols = train.dtypes[train.dtypes == np.object].index
def label_encoder(df_, qual_cols):
  df = df_.copy()
  mapping={
      'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1
  }
  for col in qual_cols :
    df[col] = df[col].map(mapping)
  return df

train = label_encoder(train, qual_cols)
test = label_encoder(test, qual_cols)
train.head()

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,10,2392,5,3,968,5,2392,2392,5,2,2003,2003,2003,386250
1,7,1352,4,2,466,4,1352,1352,5,2,2006,2007,2006,194000
2,5,900,3,1,288,3,864,900,3,1,1967,1967,1967,123000
3,5,1174,3,2,576,4,680,680,3,1,1900,2006,2000,135000
4,7,1958,4,3,936,4,1026,1026,4,2,2005,2005,2005,250000


In [12]:
print(qual_cols)

Index(['Exter Qual', 'Kitchen Qual', 'Bsmt Qual'], dtype='object')


In [13]:
def feature_eng(data_):
  data = data_.copy()
  data['Year Gap Remod'] = data['Year Remod/Add'] - data['Year Built']
  data['Car Area'] = data['Garage Area']/data['Garage Cars']
  data['2nd flr SF'] = data['Gr Liv Area'] - data['1st Flr SF']
  data['2nd flr'] = data['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0)
  data['Total SF'] = data[['Gr Liv Area',"Garage Area", "Total Bsmt SF"]].sum(axis=1)
  data['Sum Qual'] = data[["Exter Qual", "Kitchen Qual", "Overall Qual"]].sum(axis=1)
  data['Garage InOut'] = data.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1)
  return data

train = feature_eng(train)
test = feature_eng(test)

In [14]:
train.head()

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,...,Year Remod/Add,Garage Yr Blt,target,Year Gap Remod,Car Area,2nd flr SF,2nd flr,Total SF,Sum Qual,Garage InOut
0,10,2392,5,3,968,5,2392,2392,5,2,...,2003,2003,386250,0,322.666667,0,0,5752,20,0
1,7,1352,4,2,466,4,1352,1352,5,2,...,2007,2006,194000,1,233.000000,0,0,3170,15,0
2,5,900,3,1,288,3,864,900,3,1,...,1967,1967,123000,0,288.000000,0,0,2052,11,0
3,5,1174,3,2,576,4,680,680,3,1,...,2006,2000,135000,106,288.000000,494,1,2430,12,1
4,7,1958,4,3,936,4,1026,1026,4,2,...,2005,2005,250000,0,312.000000,932,1,3920,15,1


In [15]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [17]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [11]:
! pip install catboost

In [78]:
! pip install ngboost

     |████████████████████████████████| 348 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 7.0 MB/s  eta 0:00:01
  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47989 sha256=f8e97ee7ce90b5c03e24300bb141eec92d0f86d213cdd06c2657dd4ac8e71b1d
  Stored in directory: /Users/bangsumin/Library/Caches/pip/wheels/85/f5/d2/3ef47d3a836b17620bf41647222825b065245862d12aa62885
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4034 sha256=49b32fca375f13ed8c7a809d97ffaa9005e60737ef5c9559fa789bd4f5e66859
  Stored in directory: /Users/bangsumin/Library/Caches/pip/wheels/16/a2/b6/582cfdfbeeccd469504a01af3bb952fd9e7eccba40995eafea
Successfully built autograd autograd-gamma


In [18]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [19]:
nmae_score = make_scorer(NMAE, greater_is_better=False)
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [31]:
X = train.drop(['target'], axis = 1)
y = np.log1p(train.target)

target = test[X.columns]

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
continuous=['Gr Liv Area','Garage Area','Total Bsmt SF','Year Built','Year Remod/Add','Garage Yr Blt','Car Area','Total SF','Sum Qual']

In [25]:
scaler = StandardScaler()

In [26]:
train_scaler = scaler.fit_transform(train[continuous])
train[continuous] = pd.DataFrame(data=train_scaler , columns=continuous)

In [27]:
test_scaler = scaler.transform(test[continuous])
test[continuous] = pd.DataFrame(data=test_scaler, columns=continuous)

In [33]:
X.head()

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,Year Gap Remod,Car Area,2nd flr SF,2nd flr,Total SF,Sum Qual,Garage InOut
0,10,1.802547,5,3,2.435644,5,3.410442,2392,5,2,1.024446,0.888560,1.002882,0,0.777119,0,0,3.081966,2.919944,0
1,7,-0.331476,4,2,-0.188245,4,0.701583,1352,5,2,1.126848,1.087113,1.124807,1,-0.618126,0,0,0.083383,0.775729,0
2,5,-1.258955,3,1,-1.118628,3,-0.569497,900,3,1,-0.204373,-0.898415,-0.460212,0,0.237694,0,0,-1.214997,-0.939643,0
3,5,-0.696722,3,2,0.386711,4,-1.048757,680,3,1,-2.491344,1.037475,0.880958,106,0.237694,494,1,-0.776010,-0.510800,1
4,7,0.912003,4,3,2.268384,4,-0.147540,1026,4,2,1.092714,0.987837,1.084165,0,0.611142,932,1,0.954389,0.775729,1


In [34]:
y.head()

0    12.864243
1    12.175619
2    11.719948
3    11.813037
4    12.429220
Name: target, dtype: float64

In [35]:
target.head()

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,Year Gap Remod,Car Area,2nd flr SF,2nd flr,Total SF,Sum Qual,Garage InOut
0,9,0.587795,4,2,1.045297,5,1.868476,1800,5,2,1.160982,1.087113,1.165448,0,1.217996,0,0,1.398021,2.062258,0
1,6,-0.885501,3,1,-1.369518,3,-0.001679,1082,3,1,-0.852917,-1.742264,-1.232401,2,-0.509204,0,0,-0.806205,-0.510800,0
2,6,0.122004,4,2,-0.324143,4,-0.850802,769,4,2,0.922045,0.739646,0.880958,0,-0.820411,804,1,-0.382315,0.346886,1
3,6,1.907196,4,3,1.264825,4,0.196277,1158,4,2,1.058580,0.938199,1.043524,0,-0.384721,1285,1,1.447959,0.346886,1
4,5,-0.971682,3,2,0.961667,3,-0.111075,1040,3,1,-0.170240,-0.848776,0.515184,0,1.093513,0,0,-0.385799,-0.939643,0


In [36]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

In [37]:
# LinearRegression
lr_pred = np.zeros(target.shape[0])
lr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    lr = LinearRegression(normalize=True)
    lr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(lr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    lr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = lr.predict(target) / 10
    lr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(lr_val)} & std = {np.std(lr_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.07992197267082862

2 FOLD Training.....
2 FOLD NMAE = 0.10075441899684914

3 FOLD Training.....
3 FOLD NMAE = 0.09384415126606502

4 FOLD Training.....
4 FOLD NMAE = 0.11936625869521211

5 FOLD Training.....
5 FOLD NMAE = 0.07888575872108324

6 FOLD Training.....
6 FOLD NMAE = 0.10972113888413008

7 FOLD Training.....
7 FOLD NMAE = 0.09625753341578097

8 FOLD Training.....
8 FOLD NMAE = 0.10260582069583268

9 FOLD Training.....
9 FOLD NMAE = 0.0990047409710199

10 FOLD Training.....
10 FOLD NMAE = 0.09754419678403105

10FOLD Mean of NMAE = 0.09779059911008328 & std = 0.011558364285689795


In [38]:
# Ridge
rg_pred = np.zeros(target.shape[0])
rg_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    rg = Ridge()
    rg.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rg.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rg_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = rg.predict(target) / 10
    rg_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(rg_val)} & std = {np.std(rg_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.0798328888062597

2 FOLD Training.....
2 FOLD NMAE = 0.10074909925018644

3 FOLD Training.....
3 FOLD NMAE = 0.09386523625975804

4 FOLD Training.....
4 FOLD NMAE = 0.11830846475591204

5 FOLD Training.....
5 FOLD NMAE = 0.0788646002688786

6 FOLD Training.....
6 FOLD NMAE = 0.10931055896935389

7 FOLD Training.....
7 FOLD NMAE = 0.09618614715225073

8 FOLD Training.....
8 FOLD NMAE = 0.10253875950469747

9 FOLD Training.....
9 FOLD NMAE = 0.09898830309326911

10 FOLD Training.....
10 FOLD NMAE = 0.09756581045224842

10FOLD Mean of NMAE = 0.09762098685128144 & std = 0.01133509112757886


In [39]:
# Lasso
ls_pred = np.zeros(target.shape[0])
ls_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ls = Lasso()
    ls.fit(tr_x, tr_y)
    
    val_pred = np.expm1(ls.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ls_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ls.predict(target) / 10
    ls_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ls_val)} & std = {np.std(ls_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.15972717911873882

2 FOLD Training.....
2 FOLD NMAE = 0.176372088774742

3 FOLD Training.....
3 FOLD NMAE = 0.17844711637738736

4 FOLD Training.....
4 FOLD NMAE = 0.20329307778738703

5 FOLD Training.....
5 FOLD NMAE = 0.17086731121688678

6 FOLD Training.....
6 FOLD NMAE = 0.17054086522952772

7 FOLD Training.....
7 FOLD NMAE = 0.1761509453957459

8 FOLD Training.....
8 FOLD NMAE = 0.16808133182958354

9 FOLD Training.....
9 FOLD NMAE = 0.1721306207081582

10 FOLD Training.....
10 FOLD NMAE = 0.17465518060907664

10FOLD Mean of NMAE = 0.1750265717047234 & std = 0.010686549343416524


In [40]:
# ElasticNet
el_pred = np.zeros(target.shape[0])
el_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    el = ElasticNet()
    el.fit(tr_x, tr_y)
    
    val_pred = np.expm1(el.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    el_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = el.predict(target) / 10
    el_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(el_val)} & std = {np.std(el_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.15535138490859307

2 FOLD Training.....
2 FOLD NMAE = 0.17280019028411547

3 FOLD Training.....
3 FOLD NMAE = 0.17537656939048027

4 FOLD Training.....
4 FOLD NMAE = 0.20349263105969875

5 FOLD Training.....
5 FOLD NMAE = 0.16787730239344992

6 FOLD Training.....
6 FOLD NMAE = 0.16647900781016547

7 FOLD Training.....
7 FOLD NMAE = 0.171640184792867

8 FOLD Training.....
8 FOLD NMAE = 0.16572275075218554

9 FOLD Training.....
9 FOLD NMAE = 0.16813377930202997

10 FOLD Training.....
10 FOLD NMAE = 0.1776130801166027

10FOLD Mean of NMAE = 0.17244868808101882 & std = 0.011884679874618897


In [41]:
# GradientBoostingRegressor
gbr_pred = np.zeros(target.shape[0])
gbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    gbr = GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate = 0.05, n_estimators = 1000)
    gbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(gbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    gbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = gbr.predict(target) / 10
    gbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(gbr_val)} & std = {np.std(gbr_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.08716013882145425

2 FOLD Training.....
2 FOLD NMAE = 0.1042672737924906

3 FOLD Training.....
3 FOLD NMAE = 0.09632454269363645

4 FOLD Training.....
4 FOLD NMAE = 0.11620465066462951

5 FOLD Training.....
5 FOLD NMAE = 0.09644463629687659

6 FOLD Training.....
6 FOLD NMAE = 0.0915269056387866

7 FOLD Training.....
7 FOLD NMAE = 0.1021989428157251

8 FOLD Training.....
8 FOLD NMAE = 0.09921721354429447

9 FOLD Training.....
9 FOLD NMAE = 0.08839011845983755

10 FOLD Training.....
10 FOLD NMAE = 0.10644879890502812

10FOLD Mean of NMAE = 0.09881832216327593 & std = 0.00843484513865909


In [42]:
# RandomForestRegressor
rf_pred = np.zeros(target.shape[0])
rf_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    rf = RandomForestRegressor(random_state = 42, criterion = 'mae')
    rf.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rf.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rf_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(rf_val)} & std = {np.std(rf_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.08576637048038882

2 FOLD Training.....
2 FOLD NMAE = 0.09543774966712838

3 FOLD Training.....
3 FOLD NMAE = 0.09529526998698347

4 FOLD Training.....
4 FOLD NMAE = 0.11959728869232016

5 FOLD Training.....
5 FOLD NMAE = 0.09026286737389123

6 FOLD Training.....
6 FOLD NMAE = 0.09298927416805783

7 FOLD Training.....
7 FOLD NMAE = 0.09078753089997983

8 FOLD Training.....
8 FOLD NMAE = 0.09393428880986966

9 FOLD Training.....
9 FOLD NMAE = 0.09577476852666517

10 FOLD Training.....
10 FOLD NMAE = 0.10459017498807228

10FOLD Mean of NMAE = 0.0964435583593357 & std = 0.008987004671227875


In [43]:
# NGBRegressor
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.07501756889575609

2 FOLD Training.....
2 FOLD NMAE = 0.10200249562318407

3 FOLD Training.....
3 FOLD NMAE = 0.0981268703460431

4 FOLD Training.....
4 FOLD NMAE = 0.10956645566069818

5 FOLD Training.....
5 FOLD NMAE = 0.08718906659422813

6 FOLD Training.....
6 FOLD NMAE = 0.09887658455809926

7 FOLD Training.....
7 FOLD NMAE = 0.09628708060577112

8 FOLD Training.....
8 FOLD NMAE = 0.09103781305239463

9 FOLD Training.....
9 FOLD NMAE = 0.08949543862615063

10 FOLD Training.....
10 FOLD NMAE = 0.10348459790117173

10FOLD Mean of NMAE = 0.0951083971863497 & std = 0.009305141776601775


In [44]:
# Catboost
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2938486	test: 184599.8961000	best: 184599.8961000 (0)	total: 51.8ms	remaining: 2m 35s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 184599.8409
bestIteration = 145

Shrink model to first 146 iterations.
1 FOLD NMAE = 0.0809748648379099

2 FOLD Training.....
0:	learn: 0.2941968	test: 182774.3370022	best: 182774.3370022 (0)	total: 606us	remaining: 1.82s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 182774.3178
bestIteration = 88

Shrink model to first 89 iterations.
2 FOLD NMAE = 0.11446639489896013

3 FOLD Training.....
0:	learn: 0.2888881	test: 196831.3033814	best: 196831.3033814 (0)	total: 522us	remaining: 1.57s
1000:	learn: 0.0673279	test: 196831.2183133	best: 196831.2179648 (827)	total: 568ms	remaining: 1.13s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 196831.218
bestIteration = 827

Shrink model to first 828 iterations.
3 FOLD NMAE = 0.09301107252739665

4 FOLD Training.....
0:	learn: 

In [45]:
from sklearn.ensemble import StackingRegressor

In [46]:
# 검증 성능 확인하기
val_list = [lr_val, rg_val, ls_val, el_val, gbr_val, rf_val, ngb_val, cb_val]
for val in val_list :
  print("{:.8f}".format(np.mean(val))) 

0.09779060
0.09762099
0.17502657
0.17244869
0.09881832
0.09644356
0.09510840
0.12483109


In [51]:
# submission 파일에 입력
sub = pd.read_csv('sample_submission.csv')
sub['target'] = np.expm1((lr_pred + ls_pred + cb_pred +ngb_pred + rf_pred + rg_pred + gbr_pred) / 7)
sub['target']

0       315343.909792
1       130641.981165
2       172504.167635
3       257523.653974
4       133232.708111
            ...      
1345    322465.797526
1346    122719.589328
1347     94761.788662
1348    187122.012723
1349    149966.776442
Name: target, Length: 1350, dtype: float64

In [53]:
sub

,id,target
0,1,315343.909792
1,2,130641.981165
2,3,172504.167635
3,4,257523.653974
4,5,133232.708111
...,...,...
1345,1346,322465.797526
1346,1347,122719.589328
1347,1348,94761.788662
1348,1349,187122.012723


In [54]:
sub.to_csv("submission.csv",index=False)